In [1]:
using LinearAlgebra
include("ncon.jl");

[6072.821755864253 6212.139658686918 5733.013710554186 5315.100085311233 5815.789015918853 5551.303257250125 5732.390701251013 6196.709270129686 5919.509313853355 5782.478204847661; 6201.519120619368 6346.5587975058215 5864.043039181654 5422.78043772892 5924.645743634979 5713.3136201590105 5848.200348686486 6375.540434050136 6091.846632548068 6000.017082043266; 6176.360269475095 6324.249641249953 5861.119431513049 5416.791944638067 5907.306932002663 5694.2721056233895 5818.551473736374 6360.497866696497 6027.141545767248 5922.487763720833; 6013.223836127469 6204.4189411144625 5731.2188426568 5325.270694463312 5821.13390628404 5542.62476897952 5691.526112731143 6196.530025128634 5905.344568230225 5783.978588589069; 6156.221902779622 6346.65834268605 5869.218780808484 5413.698227221178 5905.437584903763 5695.619206952284 5825.740596857276 6357.662069934338 6050.152671446383 5916.648252498616; 6146.485074104129 6301.680000429027 5821.132913696364 5398.8803283811785 5894.497521588974 5648.

# Multi-stage tensor decomposition

In [8]:
##### Ex4.1(b): multi-stage decomposition
d = 5; # local dimension
chi = 3; # max internal dimension
H0 = reshape(sqrt.(1:d^7),d,d,d,d,d,d,d); # initial tensor ANY SPECIFIC REASON TO CHOOSE SUCH A TENSOR?

In [9]:
# first decomposition
utemp,stemp,vtemp = svd(reshape(H0,d^2,d^5));
U0 = reshape(utemp[:,1:chi],d,d,chi);
H1 = reshape(diagm(0 => stemp[1:chi])*vtemp[:,1:chi]',chi,d,d,d,d,d);

In [10]:
size(utemp[:,1:chi]), size(U0)

((25, 3), (5, 5, 3))

In [ ]:
# second decomposition
# IMPORTANT STEP TO UNDERSTAND
utemp,stemp,vtemp = svd(reshape(permutedims(H1,[2,3,1,4,5,6]),d^2,chi*d^3));
U1 = reshape(utemp[:,1:chi],d,d,chi);
H2 = permutedims(reshape(diagm(0 => stemp[1:chi])*vtemp[:,1:chi]',chi,chi,d,d,d),[2,1,3,4,5]);

In [ ]:
# third decomposition
utemp,stemp,vtemp = svd(reshape(H2,chi^2,d^3));
U2 = reshape(utemp[:,1:chi],chi,chi,chi);
H3 = reshape(diagm(0 => stemp[1:chi])*vtemp[:,1:chi]',chi,d,d,d);

In [ ]:
# fourth decomposition
utemp,stemp,vtemp = svd(reshape(H3,chi*d,d^2));
V3 = reshape(conj(vtemp[:,1:chi]),d,d,chi);
H4 = reshape(utemp[:,1:chi]*diagm(0 => stemp[1:chi]),chi,d,chi);

In [ ]:
# check result
H0recovered = ncon(Any[U0,U1,U2,V3,H4],Any[[-1,-2,1],[-3,-4,2],[1,2,3],[-6,-7,4],[3,-5,4]]);
totErr = norm(H0 - H0recovered) / norm(H0)

# Center of orthogonality (link centered)

In [22]:
##### Ex4.2(a): set B-C link as center of orthogonality
d = 5; # index dimension
A = rand(d,d,d);
B = rand(d,d,d);
C = rand(d,d,d);
Sig = Matrix{Float64}(I, d, d); # initial link matrix

In [23]:
# generate gauge change matrices
rho1 = ncon(Any[A,A,B,B],Any[[1,2,3],[1,2,4],[3,5,-1],[4,5,-2]]);
rho2 = ncon(Any[C,C],Any[[-1,1,2],[-2,1,2]]);
d1, u1 = eigen(rho1); sq_d1 = sqrt.(abs.(d1));
d2, u2 = eigen(rho2); sq_d2 = sqrt.(abs.(d2));
X1 = u1*diagm(0 => sq_d1)*u1'; X1inv = u1*diagm(0 => 1 ./sq_d1)*u1';
X2 = u2*diagm(0 => sq_d2)*u2'; X2inv = u2*diagm(0 => 1 ./sq_d2)*u2';

In [24]:
# implement gauge change
Bprime = ncon(Any[B,X1inv],Any[[-1,-2,1],[1,-3]]);
Cprime = ncon(Any[X2inv,C],Any[[-1,1],[1,-2,-3]]);
Sig_prime = X1*Sig*X2;

In [25]:
# check result
H0 = ncon(Any[A,B,C],Any[[-1,-2,1],[1,-3,2],[2,-4,-5]]);
H1 = ncon(Any[A,Bprime,Sig_prime,Cprime],Any[[-1,-2,1],[1,-3,2],[2,3],[3,-4,-5]]);
totErr = norm(H0 - H1) / norm(H0)

1.5866748500121967e-15

In [26]:
##### Ex4.2(b): setting a link as a center of orthogonality
##### (continues from Ex4.2(a))

# perform unitary gauge change to diagonalize link matrix
utemp, Sig_pp, vtemp = svd(Sig_prime);
Bpp = ncon(Any[Bprime,utemp],Any[[-1,-2,1],[1,-3]]);
Cpp = ncon(Any[Cprime,vtemp],Any[[1,-2,-3],[1,-1]]);
Cpp = ncon(Any[vtemp,Cprime],Any[[1,-1],[1,-2,-3]]);

In [27]:
# check result
H2 = ncon(Any[A,Bpp,diagm(0 => Sig_pp),Cpp],Any[[-1,-2,1],[1,-3,2],[2,3],[3,-4,-5]]);
totErr = norm(H0 - H2) / norm(H0)

1.285302282737601e-15

In [28]:
##### Ex4.2(c): equivalence of center of orthogonality and SVD
##### (continues from Ex4.2(b))
H = ncon(Any[A,B,C],Any[[-1,-2,1],[1,-3,2],[2,-4,-5]]);
utemp,stemp,vtemp = svd(reshape(H,d^3,d^2));
UH = reshape(utemp[:,1:d],d,d,d,d);
SH = diagm(0 => stemp[1:d]);
VH = reshape(vtemp[:,1:d]',d,d,d);

In [29]:
# compare with previous tensors from orthonormal form
ErrU = norm(abs.(UH[:]) - abs.(reshape(ncon(Any[A,Bpp],Any[[-1,-2,1],[1,-3,-4]]),d^4)))
ErrS = norm(diag(SH) - Sig_pp)
ErrV = norm(abs.(VH[:]) - abs.(Cpp[:]))
# all three results should be vanishingly small!!!

1.9025207034823104e-14

# Exercise 4

In [32]:
# pb.4 : Generating the tensor H with the element-wise definintion given
d = 6 # defining the local dimension
A1 = zeros(d,d,d,d,d)
for i = 1:d
    for j = 1:d
        for k = 1:d
            for l = 1:d
                for m = 1:d
                    A1[i,j,k,l,m] = sqrt(i+2*j+3*k+4*l+5*m)
                end
            end
        end
    end
end

chi = 4;

In [33]:
# a).
H = A1
H_norm = norm(H)
H1 = H/H_norm;
H_norm

638.9366165747585

In [36]:
# b). Multi-stage tensor decomposition
# first we do the svd
u,s,vt = svd(reshape(H1, d^2, d^3)); A = reshape(u[:,1:chi], d,d,chi);

H_temp = reshape(diagm(s[1:chi])*vt[:,1:chi]', chi, d,d,d);

# now we need to do the svd again
u, s, vt = svd(reshape(H_temp, chi*d, d^2)); B = reshape(u[:, 1:chi], chi,d,chi);

H_temp = reshape(diagm(s[1:chi])*vt[:,1:chi]', chi, d,d);

C = reshape(H_temp, chi,d,d);

# now we contract the tensors using the ncon function to obtain H1
H2 = ncon(Any[A,B,C], Any[[-1,-2,1], [1,-3,2],[2,-4,-5]]);

norm(H1-H2)

1.3381420236014653e-7

In [37]:
# c). Creating a center of orthogonality
Sig = Matrix{Float64}(I, chi, chi); # initial link matrix keeping in mind the truncation. 

rho1 = ncon(Any[A,A], [[1,2, -2],[1,2, -1]]);
rho2 = ncon(Any[B,C,B,C], Any[[-2,1,2], [2,3,4], [-1,1,5], [5,3,4]]);

# performing the eigen-decomposition
d1, u1 = eigen(rho1) ; sq_d1 = sqrt.(abs.(d1));
d2, u2 = eigen(rho2) ; sq_d2 = sqrt.(abs.(d2));

# calculating X and X^{-1}
X1 = u1*diagm(0 => sq_d1)*u1'; X1inv = u1*diagm(0 => (1 ./sq_d1))*u1';
X2 = u2*diagm(0 => sq_d2)*u2'; X2inv = u2*diagm(0 => 1 ./sq_d2)*u2';

# Now we want to implement guage change
Sig_prime = X1*Sig*X2;
A_prime = ncon(Any[A, X1inv], Any[[-1,-2,1], [1, -3]]);
B_prime = ncon(Any[X2inv, B], [[-1,1], [1,-2,-3]]);


# Now we want to diagonalise Sig_prime
# perform unitary gauge change to diagonalize link matrix
utemp, Sig_pp, vtemp = svd(Sig_prime);

App = ncon(Any[A_prime,utemp],Any[[-1,-2,1],[1,-3]]);
Bpp = ncon(Any[B_prime,vtemp],Any[[1,-2,-3],[1,-1]]);

H3 = ncon(Any[App,diagm(Sig_pp),Bpp,C], Any[[-1,-2,1], [1,2], [2,-3,3], [3,-4,-5]]);
println(Sig_pp)

# Now we will perform the svd of H2 and see if 
# we get the same singulsr values

u,s_values, v = svd(reshape(H2, d^2, d^3));
println(s_values[1:chi])

norm(Sig_pp-s_values[1:chi])

[0.9999585263044181, 0.009107102023832856, 7.975716211198103e-5, 1.6129933328501848e-6]
[0.9999585263044184, 0.009107102023832848, 7.975716211195064e-5, 1.6129933326826628e-6]


3.7415990057376464e-16

# Solutions from the tutorials

In [ ]:
# (b) multi-stage decomposition
chi = 4;
utemp,stemp,vtemp = svd(reshape(H1,d^2,d^3));
A = reshape(utemp[:,1:chi],d,d,chi);
Htemp = reshape(diagm(0 => stemp[1:chi])*(vtemp[:,1:chi]'),chi,d,d,d);
utemp,stemp,vtemp = svd(reshape(Htemp,chi*d,d^2));
B = reshape(utemp[:,1:chi],chi,d,chi);
C = reshape(diagm(0 => stemp[1:chi])*(vtemp[:,1:chi]'),chi,d,d);
# check accuracy
H2 = ncon(Any[A,B,C],Any[[-1,-2,1],[1,-3,2],[2,-4,-5]]);
TrErr = norm(H1-H2)

In [ ]:
# (c) center of orthogonality at a link
# compute gauge change between A-B tensors
SigAB = Matrix{Float64}(I, chi, chi);
rhoL = ncon(Any[A,A],Any[[1,2,-1],[1,2,-2]]);
rhoR = ncon(Any[B,B,C,C],Any[[-1,1,2],[-2,1,3],[2,4,5],[3,4,5]]);
dL,uL = eigen(rhoL); sq_dL = sqrt.(abs.(dL));
dR,uR = eigen(rhoR); sq_dR = sqrt.(abs.(dR));
XL = uL*diagm(0 => sq_dL)*uL'; XLinv = uL*diagm(0 => 1 ./sq_dL)*uL';
XR = uR*diagm(0 => sq_dR)*uR'; XRinv = uR*diagm(0 => 1 ./sq_dR)*uR';
utemp,SigABp,vtemp = svd(XL*SigAB*XR);
Ap = ncon(Any[A,XLinv*utemp],Any[[-1,-2,1],[1,-3]]);
Bp = ncon(Any[B,vtemp'*XRinv],Any[[1,-2,-3],[-1,1]]);

# difference in singular values between SVD:
~,svalues,~ = svd(reshape(H2,d^2,d^3));
Serr = norm(svalues[1:chi] - SigABp)